<a href="https://colab.research.google.com/github/BenKohn2004/Saber_Box/blob/master/Virtual_Saber_Box_Compact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from os.path import exists, join, basename, splitext
from google.colab import drive
from google.colab import files
drive.mount('/content/drive', force_remount=True)

# Determines if the Sync Folder is used for the Video Clip and Runs the Entire Sync Folder
run_most_recent_clip = False
run_entire_sync_folder = False
use_youtube_link = True

print("Note: using Google CoLab")

Mounted at /content/drive
Note: using Google CoLab


In [15]:
# Leave youtube Link as '' to use saved clip
if use_youtube_link == True:
  youtube_link = 'https://youtu.be/9Ym1osNQGts?t=147'
  # youtube_link = 'https://youtu.be/G0idubG4tSw?t=1907'

# Prevents conflicting run modes.
if run_most_recent_clip + run_entire_sync_folder + use_youtube_link != 1:
  display(f'Only one of the run modes should be set to True.')
  assert False

In [16]:
# Establishes Initial Parameters

# Provides additional comment feedback while running
verbose = False
verbose_det = False
verbose_starting = False
# Crops each from to the tracking boxes
overlay_masking_boxes = False
# Adjusts the Reperesentative Dots based on camera motion implied by scoring box movement.
camera_motion_compensate = False
# Smooths the Bellguard positions
smooth_video_clip = False
# Assumes two lights if Bellguards are close to each other. Reduces dependency on Box detection.
assume_lights = True
# Allows the Use of ScoreBox Lights Detection, True detects the Lights, while False implies Lights based on bellguard position
use_scorebox_lights = True
# Tests and Removes Duplicate Frames from the video
remove_duplicate_frames = False
# Downloads out and representative_out videos
download_videos = True
# Analyzes the Action
analyze_action = True
# Allows for Simple Usage
simplified = False
# Allows for the use of Exclusion Areas
use_Exclusion_Areas = True
# If the clip is longer than duration limit in seconds than a portion is downloaded instead of the whole clip
duration_download_limit = 2400
# Allows to run the entire clip or only till the lights turn on
run_entire_video = False
# Creates an Output video of the motion tracking
create_output_video = True
# Outputs the Bounding Box Data for the clip as a CSV file
output_bbox_clip = False
# Sets the output_bbox_clip to output the Foot Data
output_foot_data = False
# Exports the Image of the ScoreBox
export_scorebox_image = False

min_torso_confidence = 0.70
bellguard_confidence = 0.40
foot_confidence = 0.60
# Provides for a higher confidence of bellguard detection
bellguard_confidence_high = 0.45
# Provides for a higher confidence of bellguard detection
bellguard_confidence_very_high = 0.50
# Provides for a higher confidence of bellguard detection
bellguard_confidence_extra_very_high = 0.55
# Allows for a different required confidence for initial detection than tracking
bellguard_tracking_det_offset = 0.15
wrist_conf_min = 3
wrist_conf_high = 4
wrist_conf_very_high = 6
wrist_conf_extra_very_high = 8
knee_conf_min = 3
# The Threshold for determining duplicate frames
duplicate_threshold_factor = 0.75
# The Threshold for determining camera motion
camera_motion_threshold_factor = 8
# Bellguard difference as a fraction of capture width to assume lights on
position_difference_ratio = .065
# Sets an upper limit for the size in kiloBytes of a file to be used with run_most_recent_clip == True
size_max = 4000

In [17]:
# Imports
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
import pandas as pd
from xml.etree import ElementTree
from PIL import Image
# from mrcnn.utils import Dataset
from matplotlib.patches import Rectangle
import time
import random
import math
import glob
import cv2
import numpy as np
from numpy import random
from numpy import zeros
from numpy import asarray
import sys
import statistics
import PIL
import torchvision
import torch
torch.set_grad_enabled(False)
import matplotlib
import matplotlib.pylab as plt
import traceback
import os
from os import listdir
import shutil
# from shutil import copyfile
from scipy import signal
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.models import load_model
from numpy import expand_dims
from numpy import mean
from IPython.display import display, Javascript, Image, HTML
import matplotlib.pyplot as pyplot
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import io
import html
from IPython import get_ipython

import imgaug  # https://github.com/aleju/imgaug (pip3 install imageaug)

if use_youtube_link == True:
  !pip install youtube_dl
  import youtube_dl

     |████████████████████████████████| 1.9MB 10.0MB/s 


In [18]:
# Downloads Darknet AlexeyAB
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1T3BQQkqk2-E-KE45ONzij0AyyYjOepet' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1T3BQQkqk2-E-KE45ONzij0AyyYjOepet" -O darknet.zip && rm -rf /tmp/cookies.txt
!unzip '/content/darknet.zip'

--2021-07-10 02:47:40--  https://docs.google.com/uc?export=download&confirm=rdRd&id=1T3BQQkqk2-E-KE45ONzij0AyyYjOepet
Resolving docs.google.com (docs.google.com)... 74.125.142.139, 74.125.142.101, 74.125.142.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-94-docs.googleusercontent.com/docs/securesc/tqhtq5f4jd6vi3m1a3lrhtd8bbm799p8/fehcfb18874aj01bfbkpq3r5c68bpms0/1625885250000/15469335548275755091/13356425160270437420Z/1T3BQQkqk2-E-KE45ONzij0AyyYjOepet?e=download [following]
--2021-07-10 02:47:40--  https://doc-08-94-docs.googleusercontent.com/docs/securesc/tqhtq5f4jd6vi3m1a3lrhtd8bbm799p8/fehcfb18874aj01bfbkpq3r5c68bpms0/1625885250000/15469335548275755091/13356425160270437420Z/1T3BQQkqk2-E-KE45ONzij0AyyYjOepet?e=download
Resolving doc-08-94-docs.googleusercontent.com (doc-08-94-docs.googleusercontent.com)... 173.194.202.132, 2607:f8b0:400e:c00::84
Connectin

In [19]:
# Copies over Weights, data file, configuration file and two name files.
%cd '/content/'

# Weights File
# Loads the file into Content
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO" -O custom-yolov4-detector_best.weights && rm -rf /tmp/cookies.txt
# Moves the file from Content to Destination
%cp custom-yolov4-detector_best.weights /content/darknet/custom-yolov4-detector_best.weights

# Object Data File
# Loads the file into Content
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mo394Soqlfyza9I-tk73D8LfzaBu2Qte' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1mo394Soqlfyza9I-tk73D8LfzaBu2Qte" -O obj_4.data && rm -rf /tmp/cookies.txt
# Moves the file from Content to Destination
%cp obj_4.data /content/darknet/cfg/obj_4.data

# Detection Config File
# Loads the file into Content
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-4fEmuJ542QdQC9w33BNTw8mAbDVWjqW' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-4fEmuJ542QdQC9w33BNTw8mAbDVWjqW" -O custom-yolov4-detector.cfg && rm -rf /tmp/cookies.txt
# Moves the file from Content to Destination
%cp custom-yolov4-detector.cfg /content/darknet/cfg/custom-yolov4-detector.cfg

# Coco Names File
# Loads the file into Content
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-3iHBsrWNuQaeSFoaaSIsb3uE4-mBRlG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-3iHBsrWNuQaeSFoaaSIsb3uE4-mBRlG" -O coco.names && rm -rf /tmp/cookies.txt
# Moves the file from Content to Destination

%cp coco.names darknet/data/coco.names
%cp coco.names darknet/data/obj.names

/content
--2021-07-10 02:47:45--  https://docs.google.com/uc?export=download&confirm=9JOy&id=1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO
Resolving docs.google.com (docs.google.com)... 74.125.142.101, 74.125.142.139, 74.125.142.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-4o-docs.googleusercontent.com/docs/securesc/b55pg61rghqoseu98cn4238opbnfdrau/dpee8puqvk1f606gcalfuid7qtu51r6h/1625885250000/15469335548275755091/00399478595713712431Z/1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO?e=download [following]
--2021-07-10 02:47:45--  https://doc-00-4o-docs.googleusercontent.com/docs/securesc/b55pg61rghqoseu98cn4238opbnfdrau/dpee8puqvk1f606gcalfuid7qtu51r6h/1625885250000/15469335548275755091/00399478595713712431Z/1mxfy7AeCbNs6-u3f606eh0NBCF4BRUzO?e=download
Resolving doc-00-4o-docs.googleusercontent.com (doc-00-4o-docs.googleusercontent.com)... 173.194.202.132, 2607:f8b0:400e:c00::84


In [20]:
%cd darknet
# from darknet import *
!python3 darknet.py

# import darknet functions to perform object detections
from darknet import *

# Loads YOLOv4 Architecture Network
network, class_names, class_colors = load_network("cfg/custom-yolov4-detector.cfg", "cfg/obj_4.data", "custom-yolov4-detector_best.weights")
width = network_width(network)
height = network_height(network)

/content/darknet


In [21]:
# Sets the directory to Content, removes existing files and adds from the GitHub
%cd /content/
# Removes the Existing Directory, if it exists
if exists('Saber_Box'):
  shutil.rmtree('Saber_Box')
# Clones the directory
!git clone https://github.com/BenKohn2004/Saber_Box.git

# Changes directory to the downloaded git
os.chdir('/content/Saber_Box/')
execfile('setup.py')

/content
Cloning into 'Saber_Box'...
remote: Enumerating objects: 399, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 399 (delta 20), reused 0 (delta 0), pack-reused 361
Receiving objects: 100% (399/399), 5.72 MiB | 11.38 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [22]:
def youtube_clip(link):
  # Uses a given youtube_link to download and store the clip to the Sync Folder
  # Returns clip title and data if the clip duration is too long to download the entire clip

  path = '/content/drive/My Drive/Sync/'
  os.chdir(path)

  # Gets the Link for the Youtube Clip
  youtube_clip = link.split('=')[0][0:-2]
  # Uses the YouTube Query to get the time in seconds of the position in the clip
  youtube_clip_position = int(link.split('=')[1])
  # Clip End Position
  youtube_end_position = int(youtube_clip_position + 10)
  
  ydl_opts = {}
  #Gets info for the YouTube clip
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download=False)
    video_title = info_dict.get('title', None)
    file_size = info_dict.get('formats',None)[0]['filesize']
    duration = info_dict.get('duration',None)
    # container = info_dict.get('formats',None)[0]['container']

  # Initializes the container for the video file from YouTube
  container = 0
  
  display(f'The video title is {video_title}.')
  path = f'./{video_title}.mp4'

  # ydl_opts.update({'outtmpl':path})
  ydl_opts.update({'outtmpl':path})

  if verbose == True:
    display(f'The clip duration is {duration}.')

  if container == 'm4a_dash':
    display(f'The youtube link is a streamed video, downloading entire stream.')

  video_title_clip = video_title + '_clip.mp4'

  if verbose == True:
    display(f'The video title is:')
    display(video_title)

  # Downloads the entire file if the video clip is short or a DASH file, i.e. streamed video clip
  if duration < duration_download_limit or container == 'm4a_dash':
    #Downloads and saves the YouTube clip if within the duration limit
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([link])

    # The video download may be .mp4 or mkv
    if exists(video_title + '.mp4'):
      video_title = video_title + '.mp4'
    if exists(video_title + '.mkv'):
      video_title = video_title + '.mkv'

    # if verbose == True:
    display(f'Duration is less than {duration_download_limit}. Downloading entire clip.')
    display(f'The video_title is {video_title}.')

    # Renames the video without spaces to accomodate ffmpeg
    os.rename(video_title, 'source_video.mp4')

    !ffmpeg -y -ss $youtube_clip_position -i source_video.mp4 -c:v libx264 -c:a aac -frames:v 100 output_video.mp4

    os.rename('output_video.mp4', video_title_clip)
    os.remove('source_video.mp4')
    duration_limit_data = [False, 'None']
  else:
    display(f'Duration is greater than {duration_download_limit}. Downloading a portion of the clip.')
    !ffmpeg $(youtube-dl -g $link | sed "s/.*/-ss $youtube_clip_position -i &/") -y -to $youtube_end_position -c copy out.mp4

    os.rename('out.mp4', video_title_clip)

    # [starting_frame, fps, total_frames] = determine_starting_frame(video_title_clip, touch_folder, True)
    os.chdir('/content/')
    frames_per_second = 25
    # start_time = starting_frame/frames_per_second
    duration_limit_data = [True, frames_per_second]

  return (video_title_clip, duration_limit_data)

In [23]:
def save_ffmpeg(source, destination, name, fps):

  os.chdir(source)
  file_name_with_destination = os.path.join(destination, name)

  if verbose == True:
    display(f'The source of the files is {source}.')
    display(f'The save destination is {destination}.')
    display(f'Name with destionation is {file_name_with_destination}.')
    display(f'The name is {name}.')

  !ffmpeg -framerate $fps -i %1d.jpg output.mp4

  file_name_with_destination = os.path.join(destination, name)
  name = name + '.mp4'

  shutil.copy('output.mp4', file_name_with_destination)

  return

In [24]:
def youtube_clip(link):
  # Uses a given youtube_link to download and store the clip to the Sync Folder
  # Returns clip title and data if the clip duration is too long to download the entire clip

  path = '/content/drive/My Drive/Sync/'
  os.chdir(path)

  # # Gets the Link for the Youtube Clip
  # youtube_clip = link.split('=')[0][0:-2]
  # Uses the YouTube Query to get the time in seconds of the position in the clip
  youtube_clip_position = int(link.split('=')[1])
  # Clip End Position
  youtube_end_position = int(youtube_clip_position + 10)
  
  ydl_opts = {}
  #Gets info for the YouTube clip
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download=False)
    video_title = info_dict.get('title', None)
    # file_size = info_dict.get('formats',None)[0]['filesize']
    duration = info_dict.get('duration',None)
    # container = info_dict.get('formats',None)[0]['container']

  # Initializes the container for the video file from YouTube
  container = 0
  
  display(f'The video title is {video_title}.')
  path = f'./{video_title}.mp4'

  # ydl_opts.update({'outtmpl':path})
  ydl_opts.update({'outtmpl':path})

  if verbose == True:
    display(f'The clip duration is {duration}.')

  if container == 'm4a_dash':
    display(f'The youtube link is a streamed video, downloading entire stream.')

  video_title_clip = video_title + '_clip.mp4'

  if verbose == True:
    display(f'The video title is:')
    display(video_title)

  # Downloads the entire file if the video clip is short or a DASH file, i.e. streamed video clip
  if duration < duration_download_limit or container == 'm4a_dash':
    #Downloads and saves the YouTube clip if within the duration limit
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([link])

    # The video download may be .mp4 or mkv
    if exists(video_title + '.mp4'):
      video_title = video_title + '.mp4'
    if exists(video_title + '.mkv'):
      video_title = video_title + '.mkv'

    # if verbose == True:
    display(f'Duration is less than {duration_download_limit}. Downloading entire clip.')
    display(f'The video_title is {video_title}.')

    # Renames the video without spaces to accomodate ffmpeg
    os.rename(video_title, 'source_video.mp4')

    !ffmpeg -y -ss $youtube_clip_position -i source_video.mp4 -c:v libx264 -c:a aac -frames:v 100 output_video.mp4
    os.rename('output_video.mp4', video_title_clip)
    os.remove('source_video.mp4')
    duration_limit_data = [False, 'None']
  else:
    display(f'Duration is greater than {duration_download_limit}. Downloading a portion of the clip.')
    !ffmpeg $(youtube-dl -g $link | sed "s/.*/-ss $youtube_clip_position -i &/") -y -to $youtube_end_position -c copy out.mp4
    os.rename('out.mp4', video_title_clip)

    # [starting_frame, fps, total_frames] = determine_starting_frame(video_title_clip, touch_folder, True)
    os.chdir('/content/')
    frames_per_second = 25
    # start_time = starting_frame/frames_per_second
    duration_limit_data = [True, frames_per_second]

  return (video_title_clip, duration_limit_data)

In [25]:
# Checks if the Folder Hierarchy is in place

display(f'Checking Folder Hierarchy...')
create_folder_hierarchy(output_bbox_clip, output_foot_data, export_scorebox_image)

ROW_Model_in_use = 'ROW_model.h5'

# Checks if the ROW model is in the Fencing Clip Folder or the top level
if exists('/content/drive/My Drive/' + ROW_Model_in_use):
  display(f'Loading the Right of Way Model from the top level directory.')
  ROW_Model_save_path = '/content/drive/My Drive/'
elif exists('/content/drive/My Drive/projects/fencing/Fencing Clips/ROW_model.h5'):
  display(f'Loading the Right of Way Model from Fencing Clips directory.')
  ROW_Model_save_path = '/content/drive/My Drive/projects/fencing/Fencing Clips/'
else:
  # wget tool
  # https://angelov.ai/post/2020/wget-files-from-gdrive/
  # Loads the ROW_Model from a shared file
  display(f'Loading the ROW_Model into Content')
  %cd '/content/'
  # !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nvgqSTFITW74gPhbYX0hFZAmsN_MSBd_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1nvgqSTFITW74gPhbYX0hFZAmsN_MSBd_" -O ROW_model.h5 && rm -rf /tmp/cookies.txt
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xy_yvOmmzHkVzS_voJAXninjFTDFVdLn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1xy_yvOmmzHkVzS_voJAXninjFTDFVdLn" -O ROW_model.h5 && rm -rf /tmp/cookies.txt
  source = 'ROW_model.h5'
  ROW_Model_save_path = '/content/'

'Checking Folder Hierarchy...'

'The Sync Acceleration directory already exists.'

'The Sync Tracked Clips directory already exists.'

'Loading the Right of Way Model from the top level directory.'

In [26]:
loop_analysis = True
# Initializes the Previous and Most Recent Files
[most_recent_file, size] = [previous_most_recent_file, size] = Most_Recent_Sync_File()
# Sets initial size to zero
size = 0
while loop_analysis == True:
  # Determines if Youtube_Link/Sync_Folder/Stored_Clip are used for analysis
  # if youtube_link != '':
  if use_youtube_link == True:
    loop_analysis = False
    [video_title_clip, duration_limit_data] = youtube_clip(youtube_link)
    display(f'The video title clip is {video_title_clip}.')
    file_names = []
    file_names.append(video_title_clip)
    # If a clip greater than the duration limit is used, then scorebox light detection is not used.   
    if duration_limit_data[0] == True:
      use_scorebox_lights = False
  elif run_most_recent_clip == True:
    # Loops until a new file is found in the folder
    display(f'Waiting for a new file...')
    while most_recent_file == previous_most_recent_file:
      # Runs only the Most Recent File in the Sync Folder
      # # Uses the Most Recent file in the Sync Folder, copies file to 999.mp4 in Left Folder
      [most_recent_file, size] = Most_Recent_Sync_File()
      file_names = []
      file_names.append(most_recent_file)
      # Imposes a 10ms time delay
      time.sleep(0.010)
    # Saves the Previous Most Recent File for Comparison
    previous_most_recent_file = most_recent_file
    display(f'Using the Most Recently Synced File {most_recent_file}.')
  elif run_entire_sync_folder == True:
    # Sets Loop Analysis to False so that the loop only runs once.
    loop_analysis = False
    file_names = []
    path = '/content/drive/My Drive/Sync/'

    # Finds every object in a directory
    for file in os.listdir(path):
      # If the object is a file then it appends it to file_names
      name = os.path.join(path, file)
      if os.path.isfile(name):
        file_names.append(file)

  display(f'The files to be run are:')
  display(file_names)

  # Runs the most recent clip or every clip in the sync folder
  for file in file_names:
    file_name_acc = file[:-4] + '_acc.csv'
    name = os.path.join('/content/drive/My Drive/Sync/Acceleration',file_name_acc)
    display(f'The name of the acceleration file is {name}.')

      
    # Only runs a video if its acceleration file does not exist and within size limits
    if not os.path.exists(name) and size < size_max:
      display(f'Processing File {file}.')

      try:
        # Processes the Video
        if run_most_recent_clip == True or run_entire_sync_folder == True or use_youtube_link == True:
          [bbox_clip, frame_count, capture_width, capture_height, clip_vector_previous, \
          clip_vector, fps, save_image_list, too_few_frames] = \
          process_video_clip(file, 'Sync', remove_duplicate_frames)
        else:
          display(f'A method of Analysis must be set to True:')
          assert Error

        if not too_few_frames:
          # Loads ROW Model
          display(f'Determining Right of Way of the Clip.')
          model_ROW = load_model(os.path.join(ROW_Model_save_path, ROW_Model_in_use))

          clip_folder = '/content/drive/My Drive/Sync/Acceleration'
          file_name_acc = file[:-4] + '_acc.csv'

          x = load_clip(clip_folder, file_name_acc, capture_width)
          pred = model_ROW.predict(x)

          pred_list = [pred[0][0], pred[0][1], pred[0][2]]
          pred_total = pred[0][0] + pred[0][1] + pred[0][2]
          normalized_pred_results = [int(pred[0][0]/pred_total*100), int(pred[0][1]/pred_total*100), int(pred[0][2]/pred_total*100)]
          max_value = max(pred_list)
          max_index = pred_list.index(max_value)
          if max_index == 0:
            touch = 'L'
          elif max_index == 1:
            touch = 'R'
          elif max_index == 2:
            touch = 'S'
          else:
            display(f'Touch determination failed.')

          if create_output_video == True:

            for i in range(len(save_image_list)):
              # Saves the Images from the List to jpg format in Save Folder
              name = '/content/Mask_RCNN/videos/save/' + str(i) + '.jpg'
              cv2.imwrite(name, save_image_list[i])

            # Saves the Images as a Video File in the Sync Folder
            create_title_card(capture_width, capture_height, normalized_pred_results, pred_total, '', 'save')
            # video_name = file[:-4] + '_tracked_' + touch + '_' + str(int(pred[0][0]/pred_total*100)) + '_' + str(int(pred[0][1]/pred_total*100)) + '_' + str(int(pred[0][2]/pred_total*100)) 
            video_name = file[:-4] + '_tracked_' + touch + '_' + str(int(pred[0][0]/pred_total*100)) + '_' + str(int(pred[0][1]/pred_total*100)) + '_' + str(int(pred[0][2]/pred_total*100)) + '_' + str(int((pred[0][0]+pred[0][1]+pred[0][2])*100)) 
            # video_name = str(video_filename[:-4]) + '.out'
            image_save_dir = '/content/Mask_RCNN/videos/save/'
            # if run_sync_folder == True or use_sync_folder == True:
            # if run_entire_sync_folder == True or run_most_recent_clip == True:
            video_save_dir = '/content/drive/My Drive/Sync/Tracked Clips/'
            # Sorts Images
            [outvid, images] = prepare_video_frames(image_save_dir, video_save_dir, video_name)
            # Creates a video from the saved images
            destination = '/content/drive/My Drive/projects/fencing/Fencing Clips/Left_Touch/'

            # Adds suffix for encoding
            video_name =  video_name + '.mp4'
            save_ffmpeg(image_save_dir, video_save_dir, video_name, fps)
          
          if output_bbox_clip == True:
            bbox_output(video_name, bbox_clip, output_foot_data)

        else:
          display(f'Too Few Frames to Analyze.')
      except Exception as inst:
        display(f'Error running {file}.')
        traceback.print_exc()
        traceback.print_stack
    else:
      if size > size_max:
        display(f'File {file} is {size} kB, which is larger than the max of {size_max} kB.')
      if not os.path.exists(name):
        # If the file has already been run and its acceleration csv is found
        display(f'The file {file} has already been run.')

[youtube] 9Ym1osNQGts: Downloading webpage
[youtube] 9Ym1osNQGts: Downloading MPD manifest


'The video title is 2020 xx T04 02 F S Individual Porec CRO ZC Juniors PODIUM DROZD RUS vs PANTIS ROU.'

[youtube] 9Ym1osNQGts: Downloading webpage
[youtube] 9Ym1osNQGts: Downloading MPD manifest
[download] Destination: ./2020 xx T04 02 F S Individual Porec CRO ZC Juniors PODIUM DROZD RUS vs PANTIS ROU.f136.mp4
[download] 100% of 86.09MiB in 00:04
[download] Destination: ./2020 xx T04 02 F S Individual Porec CRO ZC Juniors PODIUM DROZD RUS vs PANTIS ROU.mp4.f140
[download] 100% of 6.64MiB in 00:00
[ffmpeg] Merging formats into "./2020 xx T04 02 F S Individual Porec CRO ZC Juniors PODIUM DROZD RUS vs PANTIS ROU.mp4"
Deleting original file ./2020 xx T04 02 F S Individual Porec CRO ZC Juniors PODIUM DROZD RUS vs PANTIS ROU.f136.mp4 (pass -k to keep)
Deleting original file ./2020 xx T04 02 F S Individual Porec CRO ZC Juniors PODIUM DROZD RUS vs PANTIS ROU.mp4.f140 (pass -k to keep)


'Duration is less than 2400. Downloading entire clip.'

'The video_title is 2020 xx T04 02 F S Individual Porec CRO ZC Juniors PODIUM DROZD RUS vs PANTIS ROU.mp4.'

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:457: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:463: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if sys.path[0] == '':


'Determining Right of Way of the Clip.'

'/content/drive/My Drive/Sync/Acceleration/2020 xx T04 02 F S Individual Porec CRO ZC Juniors PODIUM DROZD RUS vs PANTIS ROU_clip_acc.csv'

'The last image is 98.'

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib